# 1. GAN 소개 (Introduction to GAN)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pseudo-Lab/Tutorial-Book/blob/master/book/chapters/GAN/Ch1-Introduction.ipynb)

In [19]:
from IPython.display import HTML  # 영상 변경!!!!!! -- .../embed/JOzDr2eGFcM / embed/JO... -> ...
HTML('<iframe width="560" height="315" src="https://www.youtube.com" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## 개요 (Overview)

GAN은 Generative Adversarial Networks의 약자로 우리말로는 "적대적 생성 신경망"이라고 번역되는 AI기술 중 하나입니다. 이름에서 알 수 있듯 GAN은 두 개의 네트워크를 적대적으로(adversarial) 학습시키며 실제 데이터와 비슷한 데이터를 생성(generative)해내는 모델인데 이렇듯 정해진 label값이 없기 때문에 비지도 학습 기반 생성모델로 분류됩니다. 

GAN은 구글 브레인에서 머신러닝을 연구했던 Ian Goodfellow에 의해 2014년 처음으로 신경정보처리시스템학회(NIPS)에서 제안되었고, 이후 텍스트 생성, 이미지 생성 등에 다양하게 응용되고 있습니다.

이번 1장에서는 GAN의 개념을 비롯하여 GAN 모델의 구조와 평가지표, 적용 사례 등에 대해 알아봅니다. 1.1절에서는 GAN의 등장과 개념에 대해서 설명하고 1.2절에서는 GAN 모델을 구조와 GAN의 한 종류인 CGAN 모델, 그리고 GAN 모델의 평가지표에 대해서 설명합니다. 이어 1.3절에서는 GAN을 적용한 사례들을 살펴보며 1.4절에서는 GAN이 가진 한계점을 설명하고 있습니다.

## 1.1. 개념 (Concept)

GAN은 Generator(G,생성모델/생성기)와 Discriminator(D,판별모델/판별기) 2개의 서로 다른 네트워크로 이루어져 있으며 이 두 네트워크를 적대적으로 학습시키며 목적을 달성합니다. 생성모델(G)의 목적은 진짜 분포에 가까운 가짜분포를 생성하는 것이고 판별모델(D)의 목적은 표본이 가짜분포에 속하는지 진짜분포에 속하는지를 결정하는 것입니다. 이 2가지 모델을 포함한 GAN의 궁극적인 목적은 "실제 데이터의 분포"에 가까운 데이터를 생성하는 것이며, 따라서 판별기의 확률값이 0.5에 가까울 수록 판별기가 가짜 샘플과 실제 샘플을 구별할 수 없는 최적 솔루션으로 판단하게 됩니다. 제안자 Ian Goodfellow은 논문에서 다음과 같이 '경찰과 위조지폐범'을 예시로 들어 모델의 개념을 설명하고 있습니다. (<font style='color:red'>****사진저작권시간제한이슈!!******</font>)

<img src='./pic3/GAN-ch1img112.png'>

In [18]:
![](https://github.com/Pseudo-Lab/Tutorial-Book/blob/master/book/pics/GAN-ch1img112.png?raw=true)

zsh:1: unknown file attribute: h


생성 모델은 진짜 지폐와 비슷한 가짜 지폐를 만들어 경찰을 속이려 하는 위조지폐범 팀과 같고, 반대로 판별모델은 위조지폐범이 만들어낸 가짜 지폐를 탐지하려는 경찰팀과 유사한데 이러한 경쟁이 계속됨에 따라 위조지폐범 팀은 경찰을 속이지 못한 데이터를, 경찰팀은 위조지폐범에게 속은 데이터를 각각 입력받아 적대적으로 학습하게 되는 것입니다. 이 게임에서의 경쟁은 경쟁은 위조지폐가 진짜 지폐와 구별되지 않을 때까지, 즉 주어진 표본이 실제 표본이 될 확률이 0.5에 가까운 값을 가질 때까지 계속됩니다.

다음 절에서는 GAN 모델의 구조와 그 한 갈래인 CGAN 모델, 그리고 GAN 모델의 평가지표에 대해서 살펴봅니다.

## 1.2. GAN 모델

### 1.2.1. 모델 구조

GAN의 아키텍처를 도식화하면 다음과 같습니다.

<img src='./pic3/GAN-ch1img02.png'>

        그림 1-1 GAN의 아키텍처 (출처: ...)  
        e.g.) Zou et al. 2019. Object Detection in 20 Years: A Survey

생성기(G)는 실제 데이터와 비슷한 데이터를 만들어내도록 학습되며, 판별기(D)는 실제 데이터와 G가 생성한 가짜 데이터를 구별하도록 학습됩니다. GAN의 목적함수는 다음과 같은데, 이는 게임이론 타입의 목적함수로 G와 D 2명의 플레이어가 싸우면서 서로 균형점(nash equilibrium)을 찾아가도록 하는 방식입니다. 

<img src='./pic3/GAN-ch1img03.png'>

        그림 1-2 GAN Value Function (출처: ...)  
        e.g.) Zou et al. 2019. Object Detection in 20 Years: A Survey

여기서 V(D,G)의 값은 확률값으로 도출되는데, 이 수식을 각각 D와 G의 관점에서 살펴보면 다음과 같습니다. <br>

먼저 D의 관점에서 실제 데이터(x)를 입력하면 D(x)가 커지면서 log값이 커지면서 높은 확률값이 나오도록 하고, 가짜 데이터(G(z))를 입력하면 log값이 작아짐에 따라 낮은 확률값이 나오도록 학습됩니다. 다시 말해 D는 실제 데이터와 G가 만든 가짜 데이터를 잘 구분하도록 조금씩 업데이트되는 것입니다. <br>

G에서는 Zero-Mean Gaussian 분포에서 노이즈 z를 멀티레이어 퍼셉트론에 통과시켜 샘플들을 생성하며 이 생성된 가짜 데이터 G(z)를 D에 input으로 넣었을 때 실제 데이터처럼 확률이 높게 나오도록 학습됩니다. 즉 D(G(z))값을 높도록, 그리고 전체 확률 값이 낮아지도록 하는 것이며 이는 다시 말해 G가 'D가 잘 구분하지 못하는' 데이터를 생성하도록 조금씩 업데이트되는 것입니다.  <br>

실제 학습을 진행할 때는 G와 D 두 네트워크를 동시에 학습시키지 않고 하나의 네트워크를 고정한 상태에서 다른 한 네트워크를 업데이트하는 방식으로 따로따로 업데이트합니다.

(출처 : paper_00 : Ian J.Goodfellow 논문 & 블로그들 >>> 쉽게쉽게)

### 1.2.2. CGAN

CGAN은 Conditional Generative Adversarial Networks의 약자로 생성기와 판별기가 훈련하는 동안 추가 정보를 사용해 조건이 붙는 생성적 적대 신경망으로 GAN과는 다음 2가지 측면에서 차이가 있습니다. CGAN을 이용하면 인위적으로 원하는 클래스의 데이터를 생성할 수 있으며 생성기와 판별기를 훈련하는 데에 label을 사용합니다.
 - [Dif.1] 샘플링을 어디서 해오는지.  
 - [Dif.2] 데이터셋에 라벨이 있어야 하는지.  

<img src = './pic3/GAN-ch1img89.png'>

쉽게 말하면 Generator와 Discriminator에 특정 condition을 나타내는 정보 y를 추가해주는 것인데, 이 때 y는 형태가 정해진 것은 아니기 때문에 다양한 형태를 가질 수 있습니다. 예를 들어, 필기된 숫자를 인식하는 MNIST 데이터에서 원하는 숫자를 생성하고 싶다면 숫자의 class에 해당하는 label을 추가 정보 y로 입력해주는 것입니다. 논문에서는 생성하고 싶은 숫자 class를 one-hot encoding 하였는데, MNIST 데이터셋에서는 이 class를 one-hot encoding하게 되면 10bit가 필요하기 때문에 정보 y는 10bit 크기를 가지게 됩니다. 

<font style='color:red'> (-?) CGAN의 또다른 대표적인 응용모델로는 Pix2Pix가 있는데 Pix2Pix에서는 class의 label을 다양한 형태로...|</font>

<img src='./pic3/GAN-ch1img1221.png'>

CGAN의 value 함수는 위와 같이 정의되는데 위의 GAN의 경우와 비교해 보면 Generator와 Discriminator 각각에 추가된 정보 y에 대해 조건부 확률인 점이 차이점으로 나타납니다. 아래는 CGAN의 간단한 구조로 input에서 y정보가 추가로 들어가는 것과 output으로 나오는 확률값이 y정보에 대한 조건부인 것을 확인할 수 있습니다.

<img src='./pic3/GAN-ch1img1222.png'>

출처 : paper 03

2014년 11월 몬트리올대학교에서 최초로 발표된 CGAN 논문에서는 아래와 같이 MNIST 데이터셋으로부터 원하는 숫자(0~9까지 각각)에 대한 생성 결과를 행별로 총 10행의 결과 이미지로 보여주고 있습니다. 

<img src='./pic3/GAN-ch1img1223.png'>

출처 : paper 03

### 1.2.3. 평가 지표 (Evaluation Metrics)

GAN 창시자 Ian Goodfellow는 2014년 GAN을 소개한 이후 2016년에 학습(training)방식을 향상시키는 기술을 추가로 발표했는데 논문에서 저자들은 "GAN은 object function이 없으며 이는 서로 다른 모델들의 퍼포먼스를 비교하는 것을 어렵게 만드는 요인이다"라고 언급했습니다. 즉 주어진 GAN 모델에 대해 일반적으로 합의된 평가 방식이 없다는 것인데, 이는 학습(training)이 실행되고 있는 동안 최종적인 GAN 모델을 선택할 때, GAN 모델의 활용성을 증명하기 위해 생성된 이미지를 선택할 때, GAN 모델 아키텍처 간 비교할 때 등의 경우에 이슈가 되곤 합니다.

2018년 GAN의 평가 지표의 장단점을 기술한 논문에서 Ali Borji는 "GAN 모델의 강점과 한계점을 반영하는 가장 적절한 지표에 대해서는 아직도 의견의 일치가 이루어지지는 않았다"고 언급했습니다. 이와 같이 GAN 모델은 생성 대상의 도메인 문맥에 따라 생성된 이미지의 퀄리티에 기반하여 평가되고는 했습니다.

그 이후인 2019년 Macquarie 대학교의 Hamed Alqahtani가 발표한 논문에서는 GAN 모델의 평가지표에 대한 연구를 통해 10여 개의 GAN 모델의 평가지표를 제시하였습니다. Hamed에 따르면 GAN의 평가지표는 크게 정성적인 지표와 정량적인 지표 2가지로 분류되며 일반적으로 정성적인 지표는 사람이 이미지를 평가하므로 비용이 많이 든다고 합니다.

정성적인 지표에는 Nearest Neighbor, Rating and Preference Judgement, Rapid Scene Categorization이 있으며 정량적인 지표에는 FID(Freechet Inception Distance), IS(Inception Score), Mode Score, Maximum Mean Discrepancy 등이 있습니다. 우리는 아래에서 NVIDIA의 논문에 모델 비교시 핵심적으로 사용했던 IS와 자주 쓰이는 FID 2가지에 대해서 알아보겠습니다.

<font style='color:red'>주석있음.. </font>


<!--  - 개요
 - 6개 정도 주요한 것들 : FID, IS, Geometry Score, Tournament Based Method, Discriminator Rejection Sampling, Precision and Recall
 - FID, IS에 대해 자세히 설명. -->

#### Inception Score (IS) -- 주석있음
<!-- 보충 : https://cyc1am3n.github.io/2020/03/01/is_fid.html -->

Inception Score는 <font style='colr:red'>Salimans et al.</font>에 의해 제안되었는데 GAN의 평가에 널리 쓰이는 지표입니다. 이 지표는 클래스 label과 관련하여 특징적인 속성들을 잡아내기 위해 pre-trained 신경망을 사용합니다. 아래는 IS 값을 도출하는 수식인데 샘플의 조건부 분포 p(y|x)와 모든 샘플에서 얻은 주변분포 p(y) 사이의 평균적인 KL 발산 정도(Average KL Divergence)를 측정하는 것이며 이 값이 높을수록 좋은 성능를 낸다고 해석할 수 있습니다. 하지만 IS에는 실제 샘플 대신 생성된 이미지를 사용해 계산하고 클래스 당 하나의 이미지만 생성하면 다양성이 부족하더라도 p(y)가 균등 분포에 가깝게 나오기 때문에 성능을 왜곡할 수 있다는 단점이 있습니다.

<img src='./pic3/GAN-ch1img123.png'>

#### Frechet Inception Distance (FID)
Frechet Inception Distance는 생성되는 이미지의 퀄리티 일관성을 유지하기 위해 이용되는 지표입니다. 실제 데이터의 분포를 활용하지 않는 단점을 보완하여 실제 데이터와 생성된 데이터에서 얻은 feature의 평균과 공분산을 비교하는 방식이며 FID가 낮을수록 이미지의 퀄리티가 더 좋아지는데 이는 실제 이미지와 생성된 이미지의 유사도가 높아지는 것을 말합니다. 즉 쉽게 말해 FID는 생성된 샘플들의 통계와 실제 샘플들의 통계를 비교하는 것입니다.

**<font style='color:red'>+a</font>**  
**Annealed Importance Sampling**

(https://analyticsindiamag.com/top-6-metrics-to-monitor-the-performance-of-gans/)  

(...)  일반적으로 모델을 비교할 때 샘플링하여 비교하는 것은 매우 번거롭고 그릇될 여지를 가지고 있는 방식입니다. 하지만 Annealed Importance Sampling에서는 decoder-based 모델에 대해서 log-likelihood가 측정되고, 양방향 몬테카를로 알고리즘을 통해 정확도가 검증됩니다.

이어서 다음 절에서는 GAN을 적용한 사례들에 대해서 살펴보겠습니다.

## 1.3. 적용 사례 (Use Case)

GAN은 이미지 생성을 중심으로 다양하게 응용되어 왔는데 예시를 살펴보면 다음과 같습니다.

### 1.3.1. 가짜이미지 생성 / NVIDIA 


<img src='./pic3/GAN-ch1img98.png'>

출처 : 논문07

2017년 글로벌 GPU 설계 회사 NVIDIA에서 공개한 '실존하지 않는 사람들의 이미지'가 GAN의 대표적인 적용 사례입니다. 당시 NVIDIA는 기존의 GAN 결과들보다 이미지 품질, 안정성, 다양성 등을 향상시킨 새로운 훈련 방법론을 제시하였고 그 결과 생성된 이미지는 위와 같이 사람의 눈으로는 실존 인물인지 가상 인물인지 판별하기 어려운 수준으로 나타났습니다.  

NVIDIA가 제시한 GAN의 새로운 훈련 방법론의 핵심은 Generator와 Discriminator를 둘 다 낮은 <font style='color:red'>'결과값'</font>에서 시작하여 훈련이 진행되면서 모델이 아주 서서히 <font style='color:red'>'학습'</font>하도록 새로운 레이어를 쌓아가며 점진적으로 성장시키는 것입니다. NVIDIA는 이 방식을 통해 기존보다 높은 IS Score(GAN의 평가지표)를 달성했으며 나아가 이미지 퀄리티와 다양성을 모두 고려한 새로운 평가지표를 제안하고 있습니다.

또한, 논문에서 NVIDIA는 사람뿐 아니라 침실, 화분, 소파, 버스 등의 사물도 실제와 같은 이미지로 만들어낼 수 있음을 보여주고 있으며 이는 저해상도의 사진을 고해상도로 만드는 등 손상된 이미지를 복원할 때에도 활용될 수 있습니다.

### 1.3.2. 가짜 오바마 연설 영상 / University of Washington

<img src='./pic3/GAN-ch1img99.png'>

출처 : 논문05

2017년 워싱턴대학교(University of Washington)에서는 GAN을 이용하여 버락 오바마(Barack Obama) 전 미국 대통령의 가짜 연설 영상을 만들어 발표했습니다. 이 영상은 오바마 전 대통령의 과거 연설 영상들로부터 음성을 따고, 이 음성에 맞는 입모양을 만들어 합성한 것으로 가짜입니다. 논문에서 저자는 먼저 오디오 인풋을 시간에 따라 달라지는 입모양으로 변환한 후 진짜같은 입모양을 생성하고, 이를 대상(타겟) 비디오의 입모양 부분에 삽입하여 생성했다고 밝혔습니다. 이렇게 최종 합성 전 입모양 시퀀스와 대상 비디오를 일치시키고 타이밍을 다시 맞추어 머리 움직임과 인풋 스피치가 자연스럽게 나타날 수 있도록 한 것입니다. 영상은 <b><a href='https://www.youtube.com/watch?v=AmUC4m6w1wo&feature=youtu.be'>링크</a></b>에서 시청할 수 있습니다.

### 1.3.3. Eye In-Painting / Facebook

**<font style='color:red'>사진 택일?</font>**

<img src='./pic3/GAN-ch1img97.png'>

<img src='./pic3/GAN-ch1img95.png'>

출처 : paper 09

2017년 12월 Facebook은 ExGAN 기술을 개발하여 사람이 눈을 감은 사진에서 눈을 뜬 사람의 모습의 사진으로 바꾸는 과정을 공개했습니다. Real Eye Opener라는 명칭으로 불리는 이 작업은 GAN을 통해 진짜같은 가짜 눈을 만들어 눈을 감은 사진에 합성시킨 이미지입니다. 특정 장소나 다시 찍을 수 없는 사진에서 눈을 감고 있는 모습을 보정할 때에 활용할 수 있는 것입니다.

<h4 style='color:red'>+a.</h4>

 - MIT 텍스트 생성(풍경사진 & 짧은 시)  
 - 패션 테크 스타트업 VUE : 고객 개개인의 의상 이미지 제공

## 1.4. 한계점

지금까지 GAN이란 무엇인지, GAN 모델의 내부와 성능 평가 방식, 그리고 GAN을 적용한 사례들에 대해서 살펴보았습니다. 이렇듯 유용해 보이는 GAN 모델 역시 초기부터 한계점을 가지고 있어 왔는데 이를 살펴보면 아래와 같습니다.

GAN은 기술적으로 고해상도 이미지를 생성할 수 없다는 점과 학습이 불안정하다는 점을 한계점으로 가지고 있습니다. 이러한 한계점들을 극복하고 다양하게 응용되면서 처음 Ian Goodfellow에 의해 제안된 Vanilla GAN을 시작으로 DCGAN, SRGAN, CycleGAN 등의 GAN 모델이 개발되어 왔습니다.  

사용성 측면에서는 위의 예시들에서처럼 진짜같은 가짜를 생성하는 것이 활용도가 높은 반면 그만큼의 악용 가능성도 존재합니다. 진짜와 가짜를 구별하기 힘들다는 점을 이용한 딥페이크 기술로 만든 포르노 영상이 대표적인 예시인데, 유명인사들의 이미지를 포르노와 합성하여 배포하는 것입니다. 더하여 GAN을 이용하면 이러한 문제가 되는 데이터들을 빠르게 많이 만들어낼 수 있기에 디지털 성범죄 등 윤리적인 이슈도 수반합니다.

이 외에도, GAN을 통해 생성된 미디어의 지식재산권 이슈, 가짜 이미지를 이용한 사기 등 여러가지 법적, 윤리적인 범주의 이슈가 존재하는데 이는 GAN의 기술적 발전에 따른 제도적 대안이 수반되어야 함을 시사합니다.

**<font style='color:red'>+a?!</font>**  